# Infant resting state fMRI preprocessing
This notebook contains preprocessing tailored to infant resting state fMRI collected in 5-8 month olds. 

The processing steps for the fMRI broadly include:
* Slice-time correction
* Rigid realignment
* Co-registration to the sMRI (T2-weighted structural MRI)
* Co-registration to template
* De-noising to remove:
    - Mean timeseries for that voxel
    - Component noise associated with white matter and CSF- delete the GM and smooth what is left
    - Component noise associated with background signal - delete brain and smooth what's left
    - Component noise from the averaged timeseries
    - motion regressors
    - Motion derivatives (lagged 6 times)
    - Squared derivatives (lagged 6 times) as an exploratory
* Bandpass filtering

In [ ]:
#import packages
from os import listdir, makedirs
from os.path import isdir
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode, JoinNode        # pypeline engine
from nipype.interfaces.nipy.preprocess import Trim
from nipype.interfaces.ants import N4BiasFieldCorrection
from nipype.interfaces.fsl import SliceTimer, MCFLIRT, FLIRT, BET
from nipype.interfaces.fsl.utils import Reorient2Std, MotionOutliers
from nipype.interfaces.fsl.maths import ApplyMask, MeanImage
from nipype.interfaces.freesurfer import Resample, Binarize
from nipype.algorithms.confounds import CompCor
from pandas import DataFrame, Series,read_csv

#set output file type for FSL to NIFTI_GZ
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# Set study variables
#studyhome = '/Users/catcamacho/Box/SNAP/BABIES'
studyhome = '/home/camachocm2/Analysis/SNAP'
raw_data = studyhome + '/raw'
output_dir = studyhome + '/processed/preproc'
workflow_dir = studyhome + '/workflows'
subjects_info = read_csv(studyhome + '/misc/rest_subjects_info.csv',index_col=None, dtype={'subject_id':str})
subjects_info['subject_id'] = subjects_info['subject_id'].apply(lambda x: x.zfill(4))
subjects_list = subjects_info['subject_id'].tolist()

template_brain = studyhome + '/templates/6mo_T2w_template_2mm.nii.gz'
template_mask = studyhome + '/templates/6mo_T2w_template_2mm_mask.nii.gz'
template_gmmask = studyhome + '/templates/6mo_T2w_template_2mm_gm.nii.gz'
template_nongm = studyhome + '/templates/6mo_T2w_template_2mm_nongm.nii.gz'
template_nonbrain = studyhome + '/templates/6mo_T2w_template_2mm_nonbrain.nii.gz'
full_image = studyhome + '/templates/6mo_T2w_template_2mm_fullimage.nii.gz'

proc_cores = 4 # number of cores of processing for the workflows

vols_to_trim = 4
interleave = False
TR = 2.5 # in seconds
slice_dir = 3 # 1=x, 2=y, 3=z
resampled_voxel_size = (2,2,2)
fwhm = 4 #fwhm for smoothing with SUSAN
anat_type='t2'

In [ ]:
## File handling Nodes

# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name='infosource')
infosource.iterables = ('subject_id', subjects_list)

# Datasink- where our select outputs will go
substitutions = [('_subject_id_', '')]
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
datasink.inputs.substitutions = substitutions

## Preprocess T2w anatomical images
These nodes and workflow (anat_preprocflow) performs N4 bias correction and skullstripping.

In [ ]:
## File handling nodes

template={'anat': raw_data + '/%s*/t2w*.nii.gz'}
selectfiles = Node(DataGrabber(sort_filelist=True,
                               template = raw_data + '/%s*/t2w*.nii.gz',
                               field_template = template,
                               base_directory=raw_data,
                               infields=['subject_id'],
                               template_args={'anat':[['subject_id']]}),
                   name='selectfiles')

n4biascorr = Node(N4BiasFieldCorrection(dimension=3,
                                        output_image='{0}_nucorrect.nii.gz'.format(anat_type)), 
                  name='n4biascorr')

skullstrip = Node(BET(out_file='{0}_nucorrect_strip.nii.gz'.format(anat_type)), name='skullstrip')

In [ ]:
anat_preprocflow = Workflow(name='anat_preprocflow')
anat_preprocflow.connect([(infosource,selectfiles, [('subject_id','subject_id')]),
                          (selectfiles, n4biascorr, [('anat','input_image')]),
                          (n4biascorr, skullstrip, [('output_image','in_file')]),
                          
                          (n4biascorr, datasink, [('output_image','nu_corrected_anat')]),
                          (skullstrip, datasink, [('out_file','skullstripped_anat')])
                         ])

anat_preprocflow.base_dir = workflow_dir
anat_preprocflow.write_graph(graph2use='flat')
anat_preprocflow.run('MultiProc', plugin_args={'n_procs': 10, 'memory_gb':30})

## Preprocess fMRI resting state data
These nodes and workflow (preprocflow) perform basic preprocessing to align the functional volumes into a common space.
1. Reorient images to standard space
2. Reslice the structural image to 2mm isotropic
3. Functional image slice time correction
4. Rigid realignment to first volume of functional image
5. Coregistration of functional images to structural image
6. Coregistration of functional images to template image
7. Trim first 4 volumes of the functional images to remove pre-steady-state images

In [ ]:
## File handling Nodes

# Data grabber- select sMRI
anat_template = {'struct': output_dir + '/skullstripped_anat/{subject_id}/t2_nucorrect_strip.nii.gz'}
selectanat = Node(SelectFiles(anat_template), name='selectfiles')

# Data grabber- select fMRI
func_template = {'func':raw_data + '/%s*/rest*.nii.gz'}
selectfunc = Node(DataGrabber(sort_filelist=True,
                              template = raw_data + '/%s*/rest*.nii.gz',
                              field_template = func_template,
                              base_directory=raw_data,
                              infields=['subject_id'], 
                              template_args={'func':[['subject_id']]}), name='selectfunc')

In [ ]:
# Data QC nodes
def create_coreg_plot(epi,anat):
    import os
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    from nipype.interfaces.nipy.preprocess import Trim
    from os.path import abspath
    
    tr = Trim()
    tr.inputs.in_file = epi[0]
    tr.inputs.end_index = 1
    tr.inputs.out_file = 'firstvol.nii.gz'
    tr.run()
    
    epi = abspath('firstvol.nii.gz')
    
    coreg_filename='coregistration.png'
    display = plotting.plot_anat(epi, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'coregistration to anatomy')
    display.add_edges(anat)
    display.savefig(coreg_filename) 
    display.close()
    coreg_file = os.path.abspath(coreg_filename)
    
    return(coreg_file)

def check_mask_coverage(epi,brainmask):
    import os
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    
    from nipype.interfaces.nipy.preprocess import Trim
    from os.path import abspath
    
    tr = Trim()
    tr.inputs.in_file = epi[0]
    tr.inputs.end_index = 1
    tr.inputs.out_file = 'firstvol.nii.gz'
    tr.run()
    
    epi = abspath('firstvol.nii.gz')
    
    maskcheck_filename='maskcheck.png'
    display = plotting.plot_anat(epi, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'brainmask coverage')
    display.add_contours(brainmask,levels=[.5], colors='r')
    display.savefig(maskcheck_filename)
    display.close()
    maskcheck_file = os.path.abspath(maskcheck_filename)

    return(maskcheck_file)

make_coreg_img = Node(name='make_coreg_img',
                      interface=Function(input_names=['epi','anat'],
                                         output_names=['coreg_file'],
                                         function=create_coreg_plot))

make_checkmask_img = Node(name='make_checkmask_img',
                      interface=Function(input_names=['epi','brainmask'],
                                         output_names=['maskcheck_file'],
                                         function=check_mask_coverage))

make_checkmask_img.inputs.brainmask = template_gmmask

In [ ]:
## Nodes for preprocessing

# Reorient to standard space using FSL
reorientfunc = MapNode(Reorient2Std(), name='reorientfunc', iterfield=['in_file'])
reorientstruct = Node(Reorient2Std(), name='reorientstruct')

# Reslice- using MRI_convert 
reslice_struct = Node(Resample(voxel_size=resampled_voxel_size), name='reslice_struct')

#Slice timing correction based on interleaved acquisition using FSL
slicetime_correct = MapNode(SliceTimer(interleaved=interleave, 
                                       slice_direction=slice_dir,
                                       time_repetition=TR, 
                                       out_file='st_func.nii.gz'),
                            name='slicetime_correct', iterfield=['in_file'])
# Rigid realignment
realign = MapNode(MCFLIRT(save_plots=True,out_file='rest_moco.nii.gz'), name='realign', iterfield=['in_file'])

# Registration- using FLIRT
# The BOLD image is 'in_file', the anat is 'reference', the output is 'out_file'
firstvol = MapNode(Trim(end_index=1), name='firstvol',iterfield=['in_file'])
coreg1 = MapNode(FLIRT(), name='coreg1', iterfield=['in_file'])
coreg2 = MapNode(FLIRT(apply_xfm=True), name='coreg2', iterfield=['in_file','in_matrix_file'])

# Registration
register_template = Node(FLIRT(reference=template_brain, 
                               out_file='preproc_anat.nii.gz'), 
                         name='register_template')

xfmFUNC = MapNode(FLIRT(reference=template_brain,apply_xfm=True), 
                  name='xfmFUNC', iterfield=['in_file'])

trim = MapNode(Trim(begin_index=4, out_file='realigned_func.nii.gz'), name='trim', iterfield=['in_file'])

In [ ]:
## Preprocessing Workflow

preprocflow = Workflow(name='preprocflow')
preprocflow.connect([(infosource,selectanat,[('subject_id','subject_id')]), 
                     (infosource,selectfunc,[('subject_id','subject_id')]), 
                     (selectanat,reorientstruct,[('struct','in_file')]),
                     
                     (reorientstruct,reslice_struct,[('out_file','in_file')]),
                     (reslice_struct,coreg1,[('resampled_file','reference')]),
                     (reslice_struct,coreg2,[('resampled_file','reference')]),
                     (reslice_struct,register_template,[('resampled_file','in_file')]),
                     
                     (selectfunc,reorientfunc,[('func','in_file')]),
                     (reorientfunc,trim,[('out_file','in_file')]),
                     (trim, slicetime_correct,[('out_file','in_file')]),
                     (slicetime_correct, realign, [('slice_time_corrected_file','in_file')]),
                     (realign,firstvol,[('out_file','in_file')]),
                     (firstvol,coreg1,[('out_file','in_file')]),
                     (realign,coreg2,[('out_file','in_file')]),
                     (coreg1,coreg2,[('out_matrix_file', 'in_matrix_file')]),
                     (register_template,xfmFUNC,[('out_matrix_file','in_matrix_file')]),
                     (coreg2,xfmFUNC,[('out_file','in_file')]),
                     
                     (reorientstruct,make_coreg_img,[('out_file','anat')]),
                     (coreg1,make_coreg_img,[('out_file','epi')]),
                     (xfmFUNC,make_checkmask_img,[('out_file','epi')]),
                     (make_checkmask_img,datasink,[('maskcheck_file','maskcheck_image')]),
                     (make_coreg_img,datasink,[('coreg_file','coreg_image')]),
                   
                     (realign, datasink,[('par_file','motion_parameters')]),
                     (register_template,datasink,[('out_file','proc_struct')]),
                     (xfmFUNC, datasink, [('out_file','registered_func')])
                    ])
preprocflow.base_dir = workflow_dir
preprocflow.write_graph(graph2use='flat')
preprocflow.run('MultiProc', plugin_args={'n_procs': proc_cores})

## Create Nuissance Regressors
These nodes and workflow creates both the subject specific and general nuissance regressors needed for preprocessing the rest data per the process developed by David Montez. 

In [ ]:
# Data grabber- select fMRI
func_template = {'func':output_dir + '/registered_func/%s/*/realigned_func.nii.gz'}
selectfunc = Node(DataGrabber(sort_filelist = True,
                              template = output_dir + '/registered_func/%s/*/realigned_func.nii.gz',
                              field_template = func_template,
                              base_directory = output_dir,
                              infields=['subject_id'], 
                              template_args = {'func':[['subject_id']]}), name='selectfunc')

# select motion params
mot_template={'motion':output_dir + '/motion_parameters/%s/*/rest_moco.nii.gz.par'}
selectmotion = Node(DataGrabber(sort_filelist = True,
                                template = output_dir + '/motion_parameters/%s/*/rest_moco.nii.gz.par',
                                field_template = mot_template,
                                base_directory = output_dir,
                                infields = ['subject_id'], 
                                template_args = {'motion':[['subject_id']]}), name='selectmotion')     

In [ ]:
def mask_blur_func(mask, in_file):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import abspath
    from numpy import median, where
    from nipype.interfaces.fsl import ApplyMask, Smooth

    applymask = ApplyMask()
    applymask.inputs.mask_file = mask
    applymask.inputs.in_file = in_file
    applymask.inputs.out_file = 'masked_file.nii.gz'
    applymask.inputs.nan2zeros = True
    applymask.run()

    masked_file = abspath('masked_file.nii.gz')

    smooth = Smooth()
    smooth.inputs.in_file = masked_file
    smooth.inputs.fwhm = 4
    smooth.inputs.smoothed_file = 'blurred_masked_file.nii.gz'
    smooth.run()

    blurred_masked_file = abspath('blurred_masked_file.nii.gz')

    return(blurred_masked_file)

def leadlagmatrix(motion_file):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import abspath
    import numpy as np

    motion_params = np.loadtxt(motion_file, dtype=float)
    trs = motion_params.shape[0]
    params = motion_params.shape[1]

    leadlag = np.zeros((trs,params*6))
    derivatives = np.gradient(motion_params, axis=0)
    leadlagderivs = np.zeros((trs,params*6))
    derivativessq = derivatives**2
    leadlagderivssq = np.zeros((trs,params*6))

    for i in range(0,params):
        for j in range(0,6):
            leadlag[:,(i*j)+j] =  np.roll(motion_params[:,i],shift=j, axis=0)
            leadlag[:j,(i*j)+j] = 0

    for i in range(0,params):
        for j in range(0,6):
            leadlagderivs[:,(i*j)+j] =  np.roll(derivatives[:,i],shift=j, axis=0)
            leadlagderivs[:j,(i*j)+j] = 0

    for i in range(0,params):
        for j in range(0,6):
            leadlagderivssq[:,(i*j)+j] =  np.roll(derivativessq[:,i],shift=j, axis=0)
            leadlagderivssq[:j,(i*j)+j] = 0

    np.savetxt('leadlag.txt', leadlag)
    np.savetxt('derivsleadlag.txt', leadlagderivs)
    np.savetxt('derivssqleadlag.txt', leadlagderivssq)
    
    leadlagmot = abspath('leadlag.txt')
    leadlagderivsmot = abspath('derivsleadlag.txt')
    leadlagderivssqmot = abspath('derivssqleadlag.txt')
    
    return(leadlagmot, leadlagderivsmot, leadlagderivssqmot)

In [ ]:
# get scanner noise
session_noise = MapNode(Function(input_names=['mask','in_file'], 
                                 output_names=['blurred_masked_file'],
                                 function=mask_blur_func), name='session_noise', iterfield=['in_file'])
session_noise.inputs.mask=template_nonbrain

# get noise associated with WM and CSF
wmcsf_noise = MapNode(Function(input_names=['mask','in_file'], 
                               output_names=['blurred_masked_file'],
                               function=mask_blur_func), name='wmcsf_noise', iterfield=['in_file'])
wmcsf_noise.inputs.mask=template_nongm

# extract components from session nifti
comp_session_noise = MapNode(CompCor(repetition_time=TR,
                                  num_components=9,
                                  components_file='components.txt',
                                  mask_files=full_image), 
                          name='comp_session_noise', iterfield=['realigned_file'])

# extract components from WM-CSF nifti 
comp_wmcsf_noise = MapNode(CompCor(repetition_time=TR, 
                                   num_components=9,
                                   components_file='components.txt',
                                   mask_files=template_mask), 
                           name='comp_wmcsf_noise', iterfield=['realigned_file'])

# prepare leadlag motion and derivatives
prep_motion = MapNode(Function(input_names=['motion_file'], 
                               output_names=['leadlagmot','leadlagderivsmot','leadlagderivssqmot'],
                               function=leadlagmatrix), 
                      name='prep_motion', iterfield=['motion_file'])

In [ ]:
create_noise_flow = Workflow(name='create_noise_flow')
create_noise_flow.connect([(infosource,selectfunc,[('subject_id','subject_id')]),
                           (selectfunc, wmcsf_noise, [('func','in_file')]),
                           (selectfunc, session_noise, [('func','in_file')]),
                           (wmcsf_noise, comp_wmcsf_noise, [('blurred_masked_file','realigned_file')]),
                           (session_noise, comp_session_noise, [('blurred_masked_file','realigned_file')]),
                           (wmcsf_noise, datasink, [('blurred_masked_file','wmcsf_noise_file')]),
                           (session_noise, datasink, [('blurred_masked_file','session_noise_file')]),
                           (comp_wmcsf_noise, datasink, [('components_file','subject_wmcsf_comp_noise')]),
                           (comp_session_noise, datasink, [('components_file','subject_session_comp_noise')]),
                           
                           (infosource,selectmotion,[('subject_id','subject_id')]),
                           (selectmotion, prep_motion, [('motion','motion_file')]),
                           (prep_motion, datasink, [('leadlagmot','leadlagmotion'),
                                                    ('leadlagderivsmot','leadlagderivsmotion'),
                                                    ('leadlagderivssqmot','leadlagderivs_squaremotion')])
                          ])
create_noise_flow.base_dir = workflow_dir
create_noise_flow.write_graph(graph2use='flat')
create_noise_flow.run('MultiProc', plugin_args={'n_procs': 6})

In [ ]:
from glob import glob
from nipype import config, logging
config.enable_debug_mode()
logging.update_logging(config)
from os import mkdir, remove
from os.path import abspath
from nibabel import load, save, Nifti1Image
import numpy as np
from subprocess import check_call

mkdir(output_dir + '/mean_func')
mf_output_dir = output_dir + '/mean_func'

in_files = glob(output_dir + '/registered_func/*/*/realigned_func.nii.gz')
    
image = load(in_files[0])
data = image.get_data()
data = np.expand_dims(data,4)
maxt = data.shape[3]

for a in range(1,len(in_files)):
    tempimg = load(in_files[a])
    tempdata = tempimg.get_data()
    if tempdata.shape[3] > maxt:
        maxt=tempdata.shape[3]

files_to_avg = []        

for a in range(0,len(in_files)):
    tempimg = load(in_files[a])
    tempdata = tempimg.get_data()
    if tempdata.shape[3] < maxt:
        padn=data.shape[3]-tempdata.shape[3]
        tempdata=np.pad(tempdata,pad_width=((0,0),(0,0),(0,0),(0,padn)),
                        mode='constant',constant_values=0)
        new_img = Nifti1Image(tempdata, affine=tempimg.affine, header=tempimg.header)
        save(new_img, 'realigned_func_pad_{0}.nii.gz'.format(a))
        files_to_avg.append(abspath('realigned_func_pad_{0}.nii.gz'.format(a)))
    elif tempdata.shape[3] == maxt:
        files_to_avg.append(in_files[a])
        
mean_file=mf_output_dir+'/mean_funcs.nii.gz'

check_call(['3dMean','-prefix',mean_file,'-non_zero'] +files_to_avg)

remove('realigned_func_pad_*.nii.gz')

# extract components from averaged time series
comp_average_noise = CompCor()
comp_average_noise.inputs.realigned_file=mean_file
comp_average_noise.inputs.repetition_time=TR
comp_average_noise.inputs.num_components=9
comp_average_noise.inputs.components_file=mf_output_dir +'/components.txt' 
comp_average_noise.inputs.mask_files=template_mask
comp_average_noise.run()

## Denoising Workflow

The nodes and workflow below (denoise_flow) is designed to take the nuissance regressors created in the previous section (create_noise_flow) and perform voxel-specific denoising.  This is accomplished through the following steps:
1. Voxel-specific denoising
    - Create unique design matrix for each 3D voxel
    - Perform a GLM for that voxel
    - Project results back into 3D space
2. Bandpass filtering [0.001:0.08]
3. Concatenate and realign multiple runs

In [ ]:
means_template={'mean_func': output_dir + '/mean_func/mean_funcs.nii.gz', 
                'mean_func_components': output_dir + '/mean_func/components.txt'}
select_mean_noise = Node(SelectFiles(means_template), name='select_mean_noise')

sub_files_template={'leadlagmotion': output_dir + '/leadlagmotion/{subject_id}/_prep_motion{runnum}/leadlag.txt', 
                    'leadlagderivsmotion': output_dir + '/leadlagderivsmotion/{subject_id}/_prep_motion{runnum}/derivsleadlag.txt', 
                    'leadlagderivs_squaremotion': output_dir + '/leadlagderivs_squaremotion/{subject_id}/_prep_motion{runnum}/derivssqleadlag.txt', 
                    'motion': output_dir + '/motion_parameters/{subject_id}/_realign{runnum}/rest_moco.nii.gz.par', 
                    'func': output_dir + '/registered_func/{subject_id}/_trim{runnum}/realigned_func.nii.gz', 
                    'wmcsf': output_dir + '/subject_wmcsf_comp_noise/{subject_id}/_comp_wmcsf_noise{runnum}/components.txt', 
                    'session': output_dir + '/subject_session_comp_noise/{subject_id}/_comp_session_noise{runnum}/components.txt'}
select_sub_files=Node(SelectFiles(sub_files_template),name='select_sub_files')
select_sub_files.iterables=('runnum',['0','1'])

In [3]:
def org_shared_noise(motion, leadlagmotion, leadlagderivsmotion, leadlagderivs_squaremotion, wmcsf, session, mean_func_components):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from numpy import loadtxt, concatenate
    from pandas import DataFrame
    
    noise_list = []
    for file in [motion, leadlagmotion, leadlagderivsmotion, leadlagderivs_squaremotion]:
        mo = loadtxt(file, dtype=float, comments=None)
        length_of_file = mo.shape[0]
        noise_list.append(mo)
    for file in [wmcsf, session]:
        mo = loadtxt(file,dtype=float, skiprows=1, comments=None)
        length_of_file = mo.shape[0]
        noise_list.append(mo)
    
    mean_func_noise = loadtxt(mean_func_components,skiprows=1, comments=None)
    mean_func_noise_trim = mean_func_noise[:length_of_file,:]
    noise_list.append(mean_func_noise_trim)
    shared_noise_data = concatenate(noise_list,axis=1)
    
    col_names = ['noise_{0}'.format(a) for a in range(0,shared_noise_data.shape[1])] 
    
    shared_noise = DataFrame(shared_noise_data, columns=col_names)
    return(shared_noise)

def voxelwise_glm(func,shared_noise,mean_func,voxel_wise_noise,mask):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import abspath
    from glm.glm import GLM
    from glm.families import Gaussian
    linear_model=GLM(family=Gaussian())
    
    
    
    return(weights,pvals,sample_design_df,denoised_func)

In [4]:
motion='/Users/catcamacho/Dropbox/temp/babies_20191010/_prep_motion0/rest_moco.nii.gz.par'
leadlagmotion='/Users/catcamacho/Dropbox/temp/babies_20191010/_prep_motion0/leadlag.txt'
leadlagderivsmotion='/Users/catcamacho/Dropbox/temp/babies_20191010/_prep_motion0/derivsleadlag.txt'
leadlagderivs_squaremotion='/Users/catcamacho/Dropbox/temp/babies_20191010/_prep_motion0/derivssqleadlag.txt'
wmcsf='/Users/catcamacho/Dropbox/temp/babies_20191010/_comp_wmcsf_noise0/components.txt'
session='/Users/catcamacho/Dropbox/temp/babies_20191010/_comp_session_noise0/components.txt'
mean_func_components='/Users/catcamacho/Dropbox/temp/babies_20191010/mean_func/components.txt'

shared_noise = org_shared_noise(motion, leadlagmotion, leadlagderivsmotion, leadlagderivs_squaremotion, wmcsf, session, mean_func_components)
func = '/Users/catcamacho/Dropbox/temp/babies_20191010/_trim0/realigned_func.nii.gz'
mean_func = '/Users/catcamacho/Dropbox/temp/babies_20191010/mean_func/mean_funcs.nii.gz'
mask = '/Users/catcamacho/Box/SNAP/BABIES/templates/6mo_T2w_template_2mm_mask.nii.gz'
shared_noise.head()

(141, 6)
(141, 36)
(141, 36)
(141, 36)
(141, 9)
(141, 9)
(141, 9)


,noise_0,noise_1,noise_2,noise_3,noise_4,noise_5,noise_6,noise_7,noise_8,noise_9,...,noise_131,noise_132,noise_133,noise_134,noise_135,noise_136,noise_137,noise_138,noise_139,noise_140
0,-0.027440,0.003628,0.035594,-1.18597,0.013711,0.718586,0.719,-0.0307,0.00350,0.0325,...,0.0,-0.036840,-0.081688,-0.074396,0.166364,-0.186684,0.046491,-0.041645,0.052772,-0.133661
1,-0.027423,0.003695,0.035716,-1.12891,0.028454,0.678058,0.678,-0.0274,0.00363,0.0356,...,0.0,-0.037217,-0.066776,-0.080342,0.167456,-0.194149,0.045922,-0.021148,0.051991,-0.078587
2,-0.023436,0.004909,0.034944,-1.13915,0.142919,0.580469,0.580,-0.0274,0.00370,0.0357,...,0.0,-0.038752,-0.050624,-0.072257,0.137524,-0.200526,0.026418,-0.058535,0.087435,-0.047486
3,-0.020323,0.005587,0.035519,-1.15665,0.210243,0.437443,0.437,-0.0234,0.00491,0.0349,...,0.0,-0.019996,-0.183778,-0.019940,0.107555,-0.143408,0.007072,-0.049191,0.057967,-0.020419
4,-0.018659,0.006431,0.036198,-1.18602,0.337935,0.330649,0.331,-0.0203,0.00559,0.0355,...,0.0,-0.021187,-0.164404,0.003147,0.073808,-0.120895,-0.006119,-0.023773,0.029894,0.019897


In [7]:
from os.path import abspath
from glm.glm import GLM
from glm.families import Gaussian
from nilearn.masking import apply_mask, unmask
linear_model=GLM(family=Gaussian())
from numpy import zeros

# import data into an array that is timepoints (rows) by voxel number (columns)
func_data = apply_mask(func, mask)
mean_func_data = apply_mask(mean_func, mask)
mean_func_data = mean_func_data[:func_data.shape[0],:]
coefficients = zeros((2,func_data.shape[0]))#shared_noise.shape[1]+1))
pvals = zeros((2,func_data.shape[0]))#shared_noise.shape[1]+1))

# perform voxel-wise GLM
formula = 'signal ~ mean_signal' 
for a in range(0,shared_noise.shape[1]):
    formula = formula + ' + noise_{0}'.format(a)

formula='signal ~ mean_signal + noise_0'
for x in range(0,1):#func_data.shape[1]-1):
    shared_noise['mean_signal'] = mean_func_data[:,x]
    shared_noise['signal'] = func_data[:,x]
    linear_model.fit(shared_noise, formula=formula)
   #coefficients[:,x] = linear_model.coef_[1:]
    #pvals[:,x] = linear_model.p_values_[1:]

#coeff_image = unmask(coefficients, mask)
#pval_image = unmask(pvals, mask)
#coeff_image.to_filename('weights.nii.gz')
#pval_image.to_filename('pvals.nii.gz')
sample_design_df = shared_noise.to_csv('last_noise_mat.csv')

linear_model.summary()

Gaussian GLM Model Summary.
Name         Parameter Estimate  Standard Error
-----------------------------------------------
Intercept              -1060.07         1100.48
mean_signal                2.04            0.70
noise_0               -10983.49         1398.01


In [ ]:
compile_noise_mat = Node(Function(input_names=['leadlagmotion', 'leadlagderivsmotion', 'leadlagderivs_squaremotion', 
                                               'wmcsf', 'session', 'mean_func_components'],
                                  output_names=['shared_noise'],
                                  function=org_shared_noise), 
                         name='compile_noise_mat')
denoise_func = Node(Function(input_names=['func','shared_noise','mean_func','mask'], 
                             output_names=[],
                             function=voxelwise_glm),
                    name='denoise_func')

In [ ]:
denoise_flow = Workflow(name='denoise_flow')
denoise_flow.connect([(infosource, selectfiles,[('subject_id','subject_id')]),
                      (selectfiles, denoise, [('func','in_file')])
                     ])
denoise_flow.base_dir = workflow_dir
#denoise_flow.write_graph(graph2use='flat')
denoise_flow.run('MultiProc', plugin_args={'n_procs': proc_cores})

In [ ]:
## Pull motion info for all subjects

motion_df_file = output_dir + '/motion_summary/motionSummary.csv'

if isdir(output_dir + '/motion_summary') ==False:
    makedirs(output_dir + '/motion_summary')
    motion_df = DataFrame(columns=['meanFD','maxFD','NumCensoredVols'])
    motion_df.to_csv(motion_df_file)

def summarize_motion(motion_df_file, motion_file, vols_to_censor):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import dirname, basename
    from numpy import asarray, mean
    from pandas import DataFrame, Series, read_csv
    
    motion_df = read_csv(motion_df_file, index_col=0)
    
    motion = asarray(open(motion_file).read().splitlines()).astype(float)
    censvols = open(vols_to_censor).read().splitlines()

    fp = dirname(motion_file)
    subject = basename(fp)

    motion_df.loc[subject] = [mean(motion),max(motion),len(censvols)]
    motion_df.to_csv(motion_df_file)

    return()
    
# Remove all noise (GLM with noise params)
def create_noise_matrix(vols_to_censor,motion_params,comp_noise):
    from numpy import genfromtxt, zeros,concatenate, savetxt
    from os import path

    motion = genfromtxt(motion_params, delimiter=' ', dtype=None, skip_header=0)
    comp_noise = genfromtxt(comp_noise, delimiter='\t', dtype=None, skip_header=1)
    censor_vol_list = genfromtxt(vols_to_censor, delimiter='\t', dtype=None, skip_header=0)

    c = len(censor_vol_list)
    d = len(comp_noise)
    if c > 0:
        scrubbing = zeros((d,c),dtype=int)
        for t in range(0,c):
            scrubbing[censor_vol_list[t]][t] = 1    
        noise_matrix = concatenate([motion[:,None],comp_noise,scrubbing],axis=1)
    else:
        noise_matrix = concatenate((motion[:,None],comp_noise),axis=1)

    noise_file = 'noise_matrix.txt'
    savetxt(noise_file, noise_matrix, delimiter='\t')
    noise_filepath = path.abspath(noise_file)
    
    return(noise_filepath)

